# Métodos de clustering espectral

Los algoritmos de agrupamiento tradicionales, tales como K-means o las mixturas gaussianas, consideran por definición la existencia de clústeres con forma convexa. Es decir, se trata de conjuntos esféricos o elípticos agrupados en torno a un centro o centroide. Las técnicas de agrupamiento espectral (spectral clustering en inglés), en cambio, transforman el conjunto de datos de entrenamiento y lo representan en espacios alternativos donde se simplifica la identificación de los distintos clústeres sea cual sea su forma. Así, este tipo de técnicas novedosas permite identificar agrupamientos con formas y densidades variadas. Es necesario darse cuenta de que la identificación de los agrupamientos en estos conjuntos de datos de ejemplos mediante técnicas de agrupamiento tradicionales sería difícil o imposible

Así pues, el objetivo de un algoritmo de agrupamiento espectral podría replantearse como la búsqueda de un grupo de subconjuntos de nodos (que, en nuestro caso, representan ejemplos del conjunto de entrena- miento) tal que la probabilidad de transitar entre ellos sea mínima.



En esta práctica vamos a ver cómo funciona el clustering espectral.

Para ello, como siempre, lo primero es cargar las librerías necesarias:

In [ ]:
import numpy as np
from numpy.linalg import eig
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
from sklearn.datasets import make_circles
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import pairwise_distances
from matplotlib import pyplot as plt
import networkx as nx
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

En nuestro primer ejemplo vamos a usar un dataset muy sencillo que nos permita entender lo que hacemos:

In [ ]:
X = np.array([
    [1, 3], [2, 1], [1, 1],
    [3, 2], [7, 8], [9, 8],
    [9, 9], [8, 7], [13, 14],
    [14, 14], [15, 16], [14, 15]
])
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1])
for i, txt in enumerate(range(X.shape[0])):
    ax.annotate(txt, X[i])
plt.xlabel('Ancho')
plt.ylabel('Largo')

Tal y como acabamos de ver, el agrupamiento espectral consiste en 4 pasos básicos:

-   La obtención de la matriz de adyacencias o afinidad,
-   La obtención de la matriz Laplaciana,
-   El cálculo de los vectores y valores propios de esta última,
-   El clustering mediante K-means (u otra técnica tradicional).

Vamos a por la primera, la creación de la matriz de adyacencias o afinidad:

La forma más común de matriz en el análisis de redes sociales es una matriz cuadrada muy simple con tantas filas y columnas como actores haya en nuestro conjunto de datos. Los “elementos” o puntuaciones en las celdas de la matriz registran información sobre los vínculos entre cada par de actores.

La matriz más simple y común es la binaria. Es decir, si está presente un empate, se ingresa uno en una celda; si no hay empate, se ingresa un cero. Este tipo de matriz es el punto de partida para casi todos los análisis de redes, y se llama “matriz de adyacencia” porque representa quién está al lado, o adyacente a quién en el “espacio social” mapeado por las relaciones que hemos medido.


In [ ]:
W_dist = pairwise_distances(X,metric='euclidean')
print(W_dist)

In [ ]:
W_dist.shape

In [ ]:
plt.imshow(W_dist)

Vamos a utilizar el paquete networkx para visualizar el grafo

In [ ]:
# construímos la matriz de adyacencias o afinidad
W_ad = np.zeros_like(W_dist, np.uint8)
W_ad[W_dist < 5 ] = 1
print(W_ad)

In [ ]:
def draw_graph(G):
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

In [ ]:
G = nx.from_numpy_array(W_ad)
draw_graph(G)

`networkx` también nos permite calcular la matriz de afinidad a partir del grafo. Vamos a ver como:

Ahora tenemos que obtener la matriz Laplaciana, que recordad que se obtenía como la resta entre la matriz de adyacencia y la matriz de grado.

In [ ]:
W_ad = nx.adjacency_matrix(G)
print(W_ad)

Nos devuelve algo que no parece una matriz. Pues esto que acabamos de ver es una matriz `sparse`, que simplemente es una matriz con muy pocos elementos diferentes de 0.

Para verla en formato matriz necesitamos hacer uso del método `todense()`

In [ ]:
# matriz de grado
import numpy as np
D = np.diag(np.sum(np.array(W_ad.todense()),axis = 0)) # Le ponemos axis 1 para que nos sume por reglón
print('Matriz de grado:\n', D)

In [ ]:
# matriz laplaciana
L = D - W_ad
print('Matriz Laplaciana:\n', L)

La matriz Laplaciana es una matriz especial con ciertas propiedades que nos facilitan la vida. Una de ellas es que:

-   **Si el grafo `W` tiene `K` componentes conexas, entonces `L` tiene `K` vectores propios (*eigenvectors*) con valor propio (*eigenvalue*) igual a 0.**

En este caso, cuantos eigenvectors vamos a tener iguales a 0?

In [ ]:
# eigenvalues / eigenvectors
e, v = eig(L)
# eigenvalues
print('eigenvalues:')
print(e)
# eigenvectors
print('eigenvectors:')
print(v)

In [ ]:
plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.where(e < 10e-6)[0]
print(f'Tenemos {len(eigen_0)} componentes conexos en nuestro dataset.')
for n, i in enumerate(eigen_0):
    plt.figure()
    plt.plot(v[:, i])
    plt.title(f'{n} eigenvector con eigenvalue=0')

In [ ]:
# veamos nuestro dataset de nuevo
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], alpha=0.7, edgecolors='b')
for i, txt in enumerate(range(X.shape[0])):
    ax.annotate(txt, X[i])
plt.xlabel('Ancho')
plt.ylabel('Largo')

Si nos fijamos en las gráficas el vector propio de la matriz L (cuyos valores propios son 0), podemos ver cómo nos permiten separar correctamente los 3 clusters de nuestro dataset.

Recordad que esto es buena señal, nos indica que con esta nueva representación de los datos, un algoritmo de clustering podrá separarlos correctamente.

Vamos a comprobar si es cierto:

In [ ]:
eigen_0

In [ ]:
vectores_propios_a_utilizar = eigen_0 # en este caso queremos usar los 3, podría ser diferente

Xnew = np.array(v[:, vectores_propios_a_utilizar])
km = KMeans(init='k-means++', n_clusters=len(eigen_0))
km.fit(Xnew)
km.labels_

In [ ]:
# veamos el resultado del clustering
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=km.labels_)
for i, txt in enumerate(range(X.shape[0])):
    ax.annotate(txt, X[i])
plt.xlabel('Ancho')
plt.ylabel('Largo')

¿Y si visualizamos los datos proyectados usando los 3 vectores propios?

In [ ]:
# veamos el dataset proyectado usando los 3 vectores propios escogidos
fig, ax = plt.subplots()
ax.scatter(Xnew[:, 0], Xnew[:, 1], c=km.labels_)
plt.xlabel('Ancho')
plt.ylabel('Largo')

Ahora para k-means es facil trabajar con esta nueva representación de los datos. 

Como podes comprobar, al final, lo único que estamos haciendo es cambiar el espacio de representación de los datos por uno que nos permita agruparlos mejor.

Es básicamente el modo de funcionamiento de las redes neuronales profundas, en las que los datos van sufriendo transformaciones en cada capa hasta llegar a una representación que facilita la tarea a desarrollar.

## Ejemplo 2

Vamos a complicarlo un poco:

In [ ]:
X, clusters = make_circles(n_samples=1000, noise=.05, factor=.5, random_state=0)
plt.scatter(X[:,0], X[:,1])

Vamos a probar con el k-means, a ver qué tal...

In [ ]:
km = KMeans(n_clusters=2)
km_clustering = km.fit(X)
plt.scatter(X[:,0], X[:,1], c=km_clustering.labels_)

K-means no es capaz de hacer el agrupamiento de forma correcta.

Vamos a ver si con lo que acabamos de aprender somos capaces. Para ello, recorda que necesitamos:

- La obtención de la matriz de adyacencias o afinidad   
- La obtención de la matriz Laplaciana
- El cálculo de los vectores y valores propios de esta última
- El clustering mediante K-means (u otra técnica tradicional)

Manos a la obra:

In [ ]:
W_dist = pairwise_distances(X, metric = 'euclidean')
print('Matriz distancias:\n', W_dist)
plt.imshow(W_dist)
plt.colorbar()
plt.grid(False)

In [ ]:
# Vamos a escoger 0.5 como el umbral
W_ad = np.zeros_like(W_dist, np.uint8)
W_ad[W_dist < 0.5] = 1
print('Matriz de adyacencia:\n', W_ad)

In [ ]:
# matriz de grado
D = np.diag(np.sum(W_ad,axis = 0))
print('Matriz de grado:\n', D)

In [ ]:
# matriz laplaciana
L = D - W_ad
print('Matriz Laplaciana:\n', L)

In [ ]:
# calculamos los eigenvectors y eigenvalues
e, v = np.linalg.eig(L)

In [ ]:
plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.isclose(e, 0, atol=1e-3)
print(f'Tenemos {sum(eigen_0)} componentes conexos en nuestro dataset.')

Mirando los eigenvalues parece no que sea fácil elegir con qué componentes queremos quedarnos para hacer el clustering, ¿verdad?

¿Probamos a calcular la matriz laplaciana normalizada simétrica a ver si mejora?

In [ ]:
# calculamos la laplaciana normalizada simétrica
D = np.sum(W_ad, axis=1)
D = D**(-1./2)
I = np.diag(np.ones(D.size))
D = np.diag(D)
L = I - D.dot(W_ad)


# calculamos autovectores y autovalores
e, v = np.linalg.eig(L)

plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.where(np.isclose(e, 0, atol=1e-1))[0]
print(f'Tenemos {len(eigen_0)} componentes conexos en nuestro dataset.')
for n, i in enumerate(eigen_0):
    plt.figure()
    plt.plot(v[:, i])
    plt.title(f'{n} eigenvector con eigenvalue=0')

En este caso parece que encuentra las 6 componentes conexas! Vamos a ver si funciona el clustering:

In [ ]:
vectores_propios_a_utilizar = eigen_0
print(vectores_propios_a_utilizar)
Xnew = np.array(v[:, vectores_propios_a_utilizar])
km = KMeans(init='k-means++', n_clusters=2)
km.fit(Xnew)

In [ ]:
# veamos el resultado del clustering
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=km.labels_)

Parece que no. Vamos a ver los datos en su nuevo espacio de representación:

In [ ]:
# veamos el dataset proyectado usando los 3 vectores propios escogidos
fig, ax = plt.subplots()
ax.scatter(Xnew[:,0], Xnew[:,1], c=km.labels_, alpha=0.7)

Mirando la nueva representación de los datos, no parece que el k-means vaya a poder hacer bien el clustering, ¿verdad?

¿Se te ocurre alguna forma mejor de seguir?

¿Quizá pensando en cómo hemos construído la matriz de adyacencia? ¿Existe alguna otra forma de hacerlo?

Eso es, podemos intentarlo haciendo uso del kNN en vez del umbral ($\epsilon$).

Vamos allá:

In [ ]:
def matriz_afinidad_KNN(mSimilitud, KNN=5):
    auxM = mSimilitud.copy()
    np.fill_diagonal(auxM, 0)

    # Construimos la matriz afinidad de A a B
    mAfinidadA = np.zeros(auxM.shape)
    # utilizamos la matriz similitud para ordenar los vecinos de cada nodo por cercanía
    # `np.argsort` nos devuelve los índices que ordenan el array introducido (de forma ascendente)
    # en este caso, al tener la matriz similitud, tenemos que hacerla negativa, para que nos 
    # devuelva primero los más similares (que al ponerle el - a la matriz de similitud, son los
    # valores más pequeños)
    # Flatten simplemente lo utilizamos para convertir la matriz `KNN x n`en un vector de 1x(KNN x n)
    indices_kNN_nodo_fila = np.argsort(-auxM, axis=0)[0:KNN, :].flatten()
    # Nos creamos un array para combinar con el anterior y poder indexar los 
    # elementos pertinentes de la matriz de afinidad
    indices_kNN_nodo_col = np.tile(np.arange(auxM.shape[0]), KNN)
    mAfinidadA[indices_kNN_nodo_fila, indices_kNN_nodo_col] = 1
    np.fill_diagonal(mAfinidadA, 1)

    # Y ahora hacemos lo mismo de B a A
    mAfinidadB = np.zeros(auxM.shape)
    # Fijaos que en este caso la matriz va a ser `n x KNN` (hacemos el argsort en la dirección columnas: -->)
    indices_kNN_nodo_fila = np.repeat(np.arange(auxM.shape[0]), KNN)
    indices_kNN_nodo_col = np.argsort(-auxM, axis=1)[:, 0:KNN].flatten()
    mAfinidadB[indices_kNN_nodo_fila, indices_kNN_nodo_col] = 1
    np.fill_diagonal(mAfinidadB, 1)

    return (mAfinidadA + mAfinidadB) / 2

In [ ]:
# calculamos de nuevo la matriz de adyacencia (o afinidad), esta vez siguiendo el enfoque de los kNN
sigma = 0.1
W_sim = np.exp(-np.power(W_dist,2)/(2*sigma**2))
W_ad = matriz_afinidad_KNN(W_sim)
print(W_ad)

In [ ]:
# calculamos la laplaciana normalizada simétrica
D = np.sum(W_ad,axis=1)
D = D**(-1./2)
I = np.diag(np.ones(D.size))
D = np.diag(D)
L = I - D.dot(W_ad).dot(D)

# calculamos autovectores y autovalores
e, v = np.linalg.eig(L)

plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.where(np.isclose(e, 0, atol=1e-5))[0]
print(f'Tenemos {len(eigen_0)} componentes conexos en nuestro dataset.')
for n, i in enumerate(eigen_0):
    plt.figure()
    plt.plot(v[:, i])
    plt.title(f'{n} eigenvector con eigenvalue=0')

Parece que ahora encuentra 2 componentes conexos! Vamos a probar a hacer el clustering:

In [ ]:
vectores_propios_a_utilizar = eigen_0
print(vectores_propios_a_utilizar)
Xnew = np.array(v[:, vectores_propios_a_utilizar])
km = KMeans(init='k-means++', n_clusters=2)
km.fit(Xnew)

Veamos el resultado:

In [ ]:
# veamos el resultado del clustering
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=km.labels_, cmap='jet')

Es muy importante elegir el tipo de enfoque adecuado dependiendo de los datos. Cuando hemos construido la $W_{ad}$ mediante el método del umbral, nos hemos inventado completamente dicho umbral.

Esto no quiere decir que no exista un umbral para el cual no funcione correctamente el clustering siguiendo ese enfoque, lo que quiere decir es que si no conocemos el umbral, es posiblemente más sencillo seguir el enfoque de los kNN.

Vamos a ver los datos en su nuevo espacio:

In [ ]:
# veamos el dataset proyectado usando los 3 vectores propios escogidos
fig, ax = plt.subplots()
ax.scatter(Xnew[:,0], Xnew[:,1], c=km.labels_, alpha=0.7, cmap='jet')

Maravilloso, ¿no parece?

# Más maravilloso va a parecer aún lo siguiente:

In [ ]:
sc = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', random_state=0)
sc_clustering = sc.fit(X)
plt.scatter(X[:,0], X[:,1], c=sc_clustering.labels_, cmap='jet')

En efecto, así de rápido podemos realizar un clustering espectral utilizando las librerías existentes!

Fuente: https://towardsdatascience.com/unsupervised-machine-learning-spectral-clustering-algorithm-implemented-from-scratch-in-python-205c87271045

Ejemplo paso a paso para entender y profundizar:
-  https://towardsdatascience.com/spectral-clustering-aba2640c0d5b

Más ejemplos:

-  https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html
-  https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html

Lecturas discutiendo la similitud de k-means, clustering espectral y PCA, para quien tenga interés:

-  https://stats.stackexchange.com/a/151665
-  https://stats.stackexchange.com/a/189324
-  https://stats.stackexchange.com/a/189324 (muy completa y didáctica)
-  https://qr.ae/TW25h8


# Comparación entre Algoritmos

In [ ]:
import time
import warnings
from itertools import cycle, islice

import matplotlib.pyplot as plt
import numpy as np

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 500
seed = 30
noisy_circles = datasets.make_circles(
    n_samples=n_samples, factor=0.5, noise=0.05, random_state=seed
)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=seed)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=seed)
rng = np.random.RandomState(seed)
no_structure = rng.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
)

# ============
# Set up cluster parameters
# ============

default_base = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 3,
    "n_clusters": 3,
    "min_samples": 7,
    "xi": 0.05,
    "min_cluster_size": 0.1,
    "allow_single_cluster": True,
    "hdbscan_min_cluster_size": 15,
    "hdbscan_min_samples": 3,
    "random_state": 42,
}

datasets = [
    (
        noisy_circles,
        {
            "damping": 0.77,
            "preference": -240,
            "quantile": 0.2,
            "n_clusters": 2,
            "min_samples": 7,
            "xi": 0.08,
        },
    ),
    (
        noisy_moons,
        {
            "damping": 0.75,
            "preference": -220,
            "n_clusters": 2,
            "min_samples": 7,
            "xi": 0.1,
        },
    ),
    (
        varied,
        {
            "eps": 0.18,
            "n_neighbors": 2,
            "min_samples": 7,
            "xi": 0.01,
            "min_cluster_size": 0.2,
        },
    ),
    (
        aniso,
        {
            "eps": 0.15,
            "n_neighbors": 2,
            "min_samples": 7,
            "xi": 0.1,
            "min_cluster_size": 0.2,
        },
    ),
    (blobs, {"min_samples": 7, "xi": 0.1, "min_cluster_size": 0.2}),
    (no_structure, {}),
]

In [ ]:
plt.figure(figsize=(9 * 2 + 3, 13))
plt.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
)

plot_num = 1

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params["quantile"])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params["n_neighbors"], include_self=False
    )
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    # ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(
        n_clusters=params["n_clusters"],
        n_init="auto",
        random_state=params["random_state"],
    )
    ward = cluster.AgglomerativeClustering(
        n_clusters=params["n_clusters"], linkage="ward", connectivity=connectivity
    )
    spectral = cluster.SpectralClustering(
        n_clusters=params["n_clusters"],
        eigen_solver="arpack",
        affinity="nearest_neighbors",
        random_state=params["random_state"],
    )
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average",
        metric="cityblock",
        n_clusters=params["n_clusters"],
        connectivity=connectivity,
    )

    clustering_algorithms = (
        ("MiniBatch\nKMeans", two_means),
        ("Ward", ward),
        ("Agglomerative\nClustering", average_linkage),
        ("Spectral\nClustering", spectral)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the "
                + "connectivity matrix is [0-9]{1,2}"
                + " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning,
            )
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding"
                + " may not work as expected.",
                category=UserWarning,
            )
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, "labels_"):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(
            list(
                islice(
                    cycle(
                        [
                            "#377eb8",
                            "#ff7f00",
                            "#4daf4a",
                            "#f781bf",
                            "#a65628",
                            "#984ea3",
                            "#999999",
                            "#e41a1c",
                            "#dede00",
                        ]
                    ),
                    int(max(y_pred) + 1),
                )
            )
        )
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(
            0.99,
            0.01,
            ("%.2fs" % (t1 - t0)).lstrip("0"),
            transform=plt.gca().transAxes,
            size=15,
            horizontalalignment="right",
        )
        plot_num += 1

plt.show()